In [1]:
!pip install pyspark kafka-python delta-spark

In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.types import *
from pyspark.sql.functions import col, to_timestamp, year, month, dayofmonth, lit, udf, floor, concat, lit, window, desc
from datetime import timedelta

import os
import math
import datetime
import time
from pyspark.sql.types import StringType


In [3]:
builder = SparkSession.builder.appName("project2_debs_grand_challenge") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

#### Load data

In [4]:
columns = [
    "medallion", "hack_license", "pickup_datetime", "dropoff_datetime",
    "trip_time_in_secs", "trip_distance", "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude", "payment_type", "fare_amount",
    "surcharge", "mta_tax", "tip_amount", "tolls_amount", "total_amount"
]


df = spark.read.option("header", "false").csv("data/sorted_data.csv")
df = df.toDF(*columns)
df.printSchema()

df_sample = df.sample(withReplacement=False, fraction=0.03)

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



#### Query 0: Data cleaninig and set up

In [5]:
# Data Cleansing
df_sample = df_sample.withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"), "yyyy-MM-dd HH:mm:ss")) \
                     .withColumn("dropoff_datetime", to_timestamp(col("dropoff_datetime"), "yyyy-MM-dd HH:mm:ss"))

df_clean = df_sample.filter(
    (col("medallion").isNotNull()) & (col("medallion") != "0") &
    (col("hack_license").isNotNull()) & (col("hack_license") != "0") &
    (col("pickup_datetime").isNotNull()) &
    (col("dropoff_datetime").isNotNull()) &
    (col("trip_time_in_secs").isNotNull()) & (col("trip_time_in_secs") != 0) &
    (col("trip_distance").isNotNull()) & (col("trip_distance") != 0) &
    (col("pickup_longitude").isNotNull()) & (col("pickup_longitude") != 0.0) &
    (col("pickup_latitude").isNotNull()) & (col("pickup_latitude") != 0.0) &
    (col("dropoff_longitude").isNotNull()) & (col("dropoff_longitude") != 0.0) &
    (col("dropoff_latitude").isNotNull()) & (col("dropoff_latitude") != 0.0) &
    (col("trip_distance").cast("float") > 0) &
    (col("fare_amount").cast("float") > 0)
)

df_clean = df_clean.withColumn("trip_time_in_secs", col("trip_time_in_secs").cast("int")) \
                   .withColumn("trip_distance", col("trip_distance").cast("float")) \
                   .withColumn("fare_amount", col("fare_amount").cast("float")) \
                   .withColumn("surcharge", col("surcharge").cast("float")) \
                   .withColumn("mta_tax", col("mta_tax").cast("float")) \
                   .withColumn("tip_amount", col("tip_amount").cast("float")) \
                   .withColumn("tolls_amount", col("tolls_amount").cast("float"))

df_clean.show(5)

+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|3E7DB7D3DC9961BA3...|452B322CA3BB3132F...|2013-01-01 00:00:00|2013-01-01 00:03:00|              180|         1.52|      -73.954117|      40.778343|       -73.941818|       40.795479|         CSH|        6.0|

In [6]:
#df_clean.count()

In [7]:
# Build the Time Model
# Convert the pickup datetime to extract year, month, and day.
df_clean = df_clean.withColumn("pickup_year", year("pickup_datetime")) \
                   .withColumn("pickup_month", month("pickup_datetime")) \
                   .withColumn("pickup_day", dayofmonth("pickup_datetime"))

df_clean.show(5)

+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+-----------+------------+----------+
|           medallion|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|pickup_year|pickup_month|pickup_day|
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+-----------+------------+----------+
|3E7DB7D3DC9961BA3...|452B322CA3BB3132F...|2013-01-01 00:00:00|2013-01-01 00:03:00|              180

In [7]:
# Write Cleansed Data with File Partitioning (e.g., Parquet format)
output_path = "output/cleansed_taxi_data"
df_clean.write.partitionBy("pickup_year", "pickup_month", "pickup_day") \
    .format("parquet") \
    .mode("overwrite") \
    .save(output_path)

#### Query 1

##### Part 1

In [31]:
# 1. Compute the maximum dropoff datetime in the dataset
max_dropoff = df_clean.agg({"dropoff_datetime": "max"}).collect()[0][0]
print("Max dropoff datetime:", max_dropoff)

# 2. Define a reference time (30 minutes before max_dropoff)
ref_time = max_dropoff - timedelta(minutes=30)

# 3. Define grid cell size (here, 0.01 degrees is used as an approximation)
cell_size = 0.01

# 4. Create grid cell identifiers for pickup (start_cell) and drop-off (end_cell)
df_routes = df_clean.withColumn(
    "start_cell",
    concat(
        floor(col("pickup_latitude") / lit(cell_size)),
        lit("_"),
        floor(col("pickup_longitude") / lit(cell_size))
    )
).withColumn(
    "end_cell",
    concat(
        floor(col("dropoff_latitude") / lit(cell_size)),
        lit("_"),
        floor(col("dropoff_longitude") / lit(cell_size))
    )
)

# 5. Filter the DataFrame to include only trips whose dropoff time is within the last 30 minutes
df_last30 = df_routes.filter(col("dropoff_datetime") >= lit(ref_time))

# 6. Group by start and end cells and count the number of rides,
#    then rename the count column for clarity.
df_frequent_routes = df_last30.groupBy("start_cell", "end_cell").count() \
    .withColumnRenamed("count", "Number_of_Rides")

# 7. Order the routes by descending ride counts and take the top 10
top10_routes = df_frequent_routes.orderBy(col("Number_of_Rides").desc()).limit(10)

# 8. Show the results
top10_routes.show(truncate=False)


Max dropoff datetime: 2014-01-01 00:15:48
+----------+----------+---------------+
|start_cell|end_cell  |Number_of_Rides|
+----------+----------+---------------+
|4073_-7401|4072_-7399|3              |
|4077_-7396|4078_-7398|2              |
|4074_-7398|4077_-7395|2              |
|4076_-7399|4077_-7395|2              |
|4070_-7402|4076_-7393|1              |
|4077_-7398|4074_-7401|1              |
|4078_-7398|4080_-7397|1              |
|4076_-7398|4079_-7394|1              |
|4071_-7397|4075_-7400|1              |
|4076_-7398|4075_-7393|1              |
+----------+----------+---------------+



#### Part 2: